Katherine Kairis, kak275@pitt.edu, 11/2/2017

NEW CONTINUING -- This file continues upon the first progress report

In [1]:
import os
import glob
from bs4 import BeautifulSoup

In [2]:
file = open('data/BNC/download/Texts/G/G6/G62.xml', 'r')
text = file.read()
spoken = BeautifulSoup(text, 'xml')

In [3]:
#The BNC has both transcripts of speech and written entries. Spoken text is differentiated from written text with the 
#<stext> tag, which includes all of the speech in the file.
spoken_lines = spoken.stext

In [4]:
#The files are organized in several nested directories. Get all of the files from all of the directories.
spoken_files = []
root = 'data/BNC/download/Texts/'
for path, subdirs, files in os.walk(root):
    for name in files:
        file_name = os.path.join(path, name)

        if file_name.endswith('.xml'):
            file = open(file_name, 'r')
            text = file.read()
        
            if('<stext') in text and file_name not in spoken_files:
                spoken_files.append(file_name)

In [5]:
#Try getting the words and tags in one file
file = open(file_name, 'r')
text = file.read()
xml_contents = BeautifulSoup(text, 'xml')

In [6]:
#Gets all utterances in the file
utterances = xml_contents.findAll('u')

In [7]:
#Gets the first utterance in the file
u1 = utterances[0]

In [8]:
#Gets speaker of the first utterance
u1['who']

'KSWPSUNK'

In [9]:
#Gets the first sentence of the utterance (in XML format)
segment1 = utterances[0].findAll('s')[0]

In [10]:
segment1

<s n="1"><w c5="AV0" hw="right" pos="ADV">Right</w><c c5="PUN">, </c><w c5="ITJ" hw="hello" pos="INTERJ">hello</w><c c5="PUN">, </c><w c5="ITJ" hw="yeah" pos="INTERJ">yeah</w><c c5="PUN">, </c><w c5="PNP" hw="we" pos="PRON">we</w><w c5="VBB" hw="be" pos="VERB">'re </w><w c5="AVP" hw="back" pos="ADV">back</w><c c5="PUN">.</c></s>

In [11]:
#Gets the text of the first sentence
segment1.text

"Right, hello, yeah, we're back."

In [12]:
#Get all of the words (in XML format) of the sentence
words = segment1.findAll('w')

In [13]:
words

[<w c5="AV0" hw="right" pos="ADV">Right</w>,
 <w c5="ITJ" hw="hello" pos="INTERJ">hello</w>,
 <w c5="ITJ" hw="yeah" pos="INTERJ">yeah</w>,
 <w c5="PNP" hw="we" pos="PRON">we</w>,
 <w c5="VBB" hw="be" pos="VERB">'re </w>,
 <w c5="AVP" hw="back" pos="ADV">back</w>]

In [14]:
#Get the word and tag of each word entry
for w in words:
    print(w['pos'])
    print(w.text)

ADV
Right
INTERJ
hello
INTERJ
yeah
PRON
we
VERB
're 
ADV
back


In [15]:
#Iterate through all of the files in the BNC to two nested dictionaries: one containing word tokens of the
#conversations, and the other containing (word, tag) tuples for every word in the corpus.
#The keys of the dictionaries are the file names. The values of these entries are subdictionaries. The keys of the
#subdictionary are (participant, line_number) tuples, and the values are either lists of tokens or (word, tag) tuples.
tokenize = {}
tagged = {}

for file_name in spoken_files:
    token_dict = {}
    tag_dict = {}
        
    file = open(file_name, 'r')
    text = file.read()
    xml_contents = BeautifulSoup(text, 'xml')
    
    utterances = xml_contents.findAll('u')
    
    for u in utterances:
        speaker = u['who']
        segments = u.findAll('s')
        
        for s in segments:
            line = s['n']
            #print((speaker, line))
            words = s.findAll('w')
            
            token_list = []
            tag_list = []
            
            for w in words:
                word = w.text
                tag = w['pos']
                
                token_list.append(word)
                tag_list.append((word, tag))
                
                #print((word, tag))
            token_dict[(speaker, line)] = token_list
            tag_dict[(speaker, line)] = tag_list
            
            
    tokenize[file_name] = token_dict
    tagged[file_name] = tag_dict
    

    

    

In [19]:
#Save the dictionaries as pickle files.
import pickle

In [22]:
f = open('BNC_tokenized.p', 'wb')
pickle.dump(tokenize, f, -1)
f.close()

In [23]:
f = open('BNC_tagged.p', 'wb')
pickle.dump(tagged, f, -1)
f.close()